In [6]:
import os, yaml
from datetime import datetime
from easydict import EasyDict
from glob import glob
import pickle

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader

from dataloader.bci_compet import get_dataset
from dataloader.bci_compet import BCICompet2bIV

from model.litmodel import LitModel
from model.litmodel import get_litmodel
from model.cat_conditioned import CatConditioned
from model.attn_conditioned import ATTNConditioned
from model.attn_conditioned_subj_avg import ATTNConditionedSubjAvg
from model.attn_conditioned_subj_ftr import ATTNConditionedSubjFtr


from pytorch_lightning import Trainer, seed_everything

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, cohen_kappa_score

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
### Set confings
config_name = 'bcicompet2b_config'
with open(f'configs/{config_name}.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
    args = EasyDict(config)
    

In [8]:
#### Set Device ####
# if torch.cuda.is_available():
#     os.environ['CUDA_VISIBLE_DEVICES'] = str(args.GPU_NUM)
cudnn.benchmark = True
cudnn.fastest = True
cudnn.deterministic = True

#### Set SEED ####
seed_everything(args.SEED)

#### Update configs ####
if args.downsampling != 0: args['sampling_rate'] = args.downsampling


Seed set to 42


In [12]:
CACHE_ROOT = 'cache'
TEST_ROOT = 'test_results'

args.is_test = True

def load_dataset(args, return_subject_id=False):
    datasets = {}
    for subject_id in range(0,9):
        args['target_subject'] = subject_id
        datasets[subject_id] = BCICompet2bIV(args)
    return datasets

path = os.path.join(CACHE_ROOT, f'{config_name}.pkl')

if not os.path.isfile(path):
    print('Cache miss, generating cache')
    datasets = load_dataset(args)
    with open(path, 'wb') as file:
        pickle.dump(datasets, file)
else:
    print('Loading cache')
    with open(path, 'rb') as file:
        datasets = pickle.load(file)

Loading cache


In [14]:
version = '2023-12-14_10-53-24-baseline'
subj_info = ''

# version = '2023-12-11_14-45-01-cat-2'
# subj_info = 'id'

# version = '2023-12-11_14-33-07-attn-1'
# subj_info = 'id'

# version = '2023-12-11_15-01-18-avg-0'
# subj_info = 'avg'

# version = '2023-12-11_16-33-05-ftr-1'
# subj_info = 'ftr'

ckpt_path = sorted(glob(f'{args.CKPT_PATH}/{args.task}/{version}/*.ckpt'))[-1]
print(ckpt_path)

# TODO use args from optuna histtory
model = get_litmodel(args)
#in_model = CatConditioned(args, embedding_dimension=5, combined_features_dimension=46 )
#in_model = ATTNConditioned(args, embedding_dimension=7, combined_features_dimension=39)
#in_model =  ATTNConditionedSubjAvg(args, embedding_dimension=44, combined_features_dimension=72 )
#in_model = ATTNConditionedSubjFtr(args, embedding_dimension=23, combined_features_dimension=43, subj_dim=26 )
#model = LitModel(args, in_model)
model.load_state_dict(torch.load(ckpt_path)['state_dict'], strict=False)
trainer = Trainer()

model.eval()

./checkpoints/BCICompet2b/2023-12-14_10-53-24-baseline/epoch=278-val_loss=0.495.ckpt


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LitModel(
  (model): Net(
    (backbone): MShallowConvNet(
      (temporal_conv): Conv2dWithConstraint(1, 24, kernel_size=(1, 30), stride=(1, 1), padding=same)
      (spatial_conv): Conv2dWithConstraint(24, 24, kernel_size=(3, 1), stride=(1, 1), padding=valid)
      (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (avg_pool): AvgPool2d(kernel_size=[1, 75], stride=[1, 22], padding=0)
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (classifier): classifier(
      (dense): LazyLinearWithConstraint(in_features=0, out_features=2, bias=True)
    )
  )
  (criterion): CrossEntropyLoss()
)

In [17]:

result_dict = {}

for subject_id in range(0,9):
    datasets[subject_id].return_subject_info = subj_info
    test_dataloader = DataLoader(datasets[subject_id],
                                    batch_size=args.batch_size,
                                    pin_memory=False,
                                    num_workers=args.num_workers)
    gt = datasets[subject_id].label                                   
    logits = trainer.predict(model, dataloaders=test_dataloader)
    pred = torch.cat(logits, dim=0).argmax(axis=1).detach().cpu().numpy()
    acc = accuracy_score(pred, gt)
    kappa = cohen_kappa_score(pred, gt)
    result_dict[subject_id] = {'acc': acc, 'kappa': kappa}
result_df = pd.DataFrame(result_dict).T

result_df.to_csv(f'{TEST_ROOT}/{args.task}/{version}.csv')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
result_df['acc'].mean(), result_df['kappa'].mean()

(0.73070987654321, 0.6409465020576132)